https://drive.google.com/drive/u/0/folders/1X_ag69ToGRCF32K2a099pgvvNMg0j8nq

In [33]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

In [34]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        
        
        

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                
                
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1a = min(min_price_index_1, max_price_index_1)
            larger_index_1a = max(min_price_index_1, max_price_index_1)
            remaining_prices_11a = current_df[(current_df.index > smaller_index_1a) & (current_df.index < larger_index_1a)]
            remaining_prices_11a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1a) & (Q_A_Preds.index < larger_index_1a)]
            remaining_prices_11a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1a) & (Q_B_Preds.index < larger_index_1a)]

            if len(remaining_prices_11a) > 1:
                max_price_index_1a = remaining_prices_11a_A['Price'].idxmax()
                min_price_index_1a = remaining_prices_11a_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_1a, 'Price'] < current_Q_A.loc[max_price_index_1a, 'Price']:
                    if min_price_index_1a < max_price_index_1a:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                    elif min_price_index_1a > max_price_index_1a:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                    else:
                         continue
                            
                            
                            
                            
                smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                remaining_prices_11b = current_df[(current_df.index > smaller_index_1b) & (current_df.index < larger_index_1b)]
                remaining_prices_11b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1b) & (Q_A_Preds.index < larger_index_1b)]
                remaining_prices_11b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1b) & (Q_B_Preds.index < larger_index_1b)]

                if len(remaining_prices_11b) > 1:
                    max_price_index_1b = remaining_prices_11b_A['Price'].idxmax()
                    min_price_index_1b = remaining_prices_11b_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_1b, 'Price'] < current_Q_A.loc[max_price_index_1b, 'Price']:
                        if min_price_index_1b < max_price_index_1b:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                        elif min_price_index_1b > max_price_index_1b:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                        else:
                             continue                           
                            
                            
                            
                    smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                    larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                    remaining_prices_11c = current_df[(current_df.index > smaller_index_1c) & (current_df.index < larger_index_1c)]
                    remaining_prices_11c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1c) & (Q_A_Preds.index < larger_index_1c)]
                    remaining_prices_11c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1c) & (Q_B_Preds.index < larger_index_1c)]

                    if len(remaining_prices_11c) > 1:
                        max_price_index_1c = remaining_prices_11c_A['Price'].idxmax()
                        min_price_index_1c = remaining_prices_11c_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_1c, 'Price'] < current_Q_A.loc[max_price_index_1c, 'Price']:
                            if min_price_index_1c < max_price_index_1c:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1c > max_price_index_1c:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                           
                                               
                                        
                                        
                                        
                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                        remaining_prices_11d = current_df[(current_df.index > smaller_index_1d) & (current_df.index < larger_index_1d)]
                        remaining_prices_11d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1d) & (Q_A_Preds.index < larger_index_1d)]
                        remaining_prices_11d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1d) & (Q_B_Preds.index < larger_index_1d)]

                        if len(remaining_prices_11d) > 1:
                            max_price_index_1d = remaining_prices_11d_A['Price'].idxmax()
                            min_price_index_1d = remaining_prices_11d_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_1d, 'Price'] < current_Q_A.loc[max_price_index_1d, 'Price']:
                                if min_price_index_1d < max_price_index_1d:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                elif min_price_index_1d > max_price_index_1d:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                      prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                 
                            
                            
                            
                            
                            
                            
                            
                            

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_02a = current_df[(current_df.index > smaller_index_2a) & (current_df.index < larger_index_2a)]
                remaining_prices_02a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2a) & (Q_A_Preds.index < larger_index_2a)]
                remaining_prices_02a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2a) & (Q_B_Preds.index < larger_index_2a)]

                if len(remaining_prices_02a) > 1:
                    max_price_index_2a = remaining_prices_02a_A['Price'].idxmax()
                    min_price_index_2a = remaining_prices_02a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_2a, 'Price'] < current_Q_A.loc[max_price_index_2a, 'Price']:
                        if min_price_index_2a < max_price_index_2a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_2a > max_price_index_2a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                    larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                    remaining_prices_02b = current_df[(current_df.index > smaller_index_2b) & (current_df.index < larger_index_2b)]
                    remaining_prices_02b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2b) & (Q_A_Preds.index < larger_index_2b)]
                    remaining_prices_02b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2b) & (Q_B_Preds.index < larger_index_2b)]

                    if len(remaining_prices_02b) > 1:
                        max_price_index_2b = remaining_prices_02b_A['Price'].idxmax()
                        min_price_index_2b= remaining_prices_02b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_2b, 'Price'] < current_Q_A.loc[max_price_index_2b, 'Price']:
                            if min_price_index_2b < max_price_index_2b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2b > max_price_index_2b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                        
                                  prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                        larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                        remaining_prices_02c = current_df[(current_df.index > smaller_index_2c) & (current_df.index < larger_index_2c)]
                        remaining_prices_02c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2c) & (Q_A_Preds.index < larger_index_2c)]
                        remaining_prices_02c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2c) & (Q_B_Preds.index < larger_index_2c)]

                        if len(remaining_prices_02c) > 1:
                            max_price_index_2c = remaining_prices_02c_A['Price'].idxmax()
                            min_price_index_2c= remaining_prices_02c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_2c, 'Price'] < current_Q_A.loc[max_price_index_2c, 'Price']:
                                if min_price_index_2c < max_price_index_2c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_2c > max_price_index_2c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                            
                                      prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                            larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                            remaining_prices_02d = current_df[(current_df.index > smaller_index_2d) & (current_df.index < larger_index_2d)]
                            remaining_prices_02d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2d) & (Q_A_Preds.index < larger_index_2d)]
                            remaining_prices_02d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2d) & (Q_B_Preds.index < larger_index_2d)]

                            if len(remaining_prices_02d) > 1:
                                max_price_index_2d = remaining_prices_02d_A['Price'].idxmax()
                                min_price_index_2d= remaining_prices_02d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_2d, 'Price'] < current_Q_A.loc[max_price_index_2d, 'Price']:
                                    if min_price_index_2d < max_price_index_2d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2d > max_price_index_2d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                            
                                          prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue  
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_3a = min(min_price_index_3, max_price_index_3)
                larger_index_3a = max(min_price_index_3, max_price_index_3)
                remaining_prices_03a = current_df[(current_df.index > smaller_index_3a) & (current_df.index < larger_index_3a)]
                remaining_prices_03a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3a) & (Q_A_Preds.index < larger_index_3a)]
                remaining_prices_03a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3a) & (Q_B_Preds.index < larger_index_3a)]

                if len(remaining_prices_03a) > 1:
                    max_price_index_3a = remaining_prices_03a_A['Price'].idxmax()
                    min_price_index_3a = remaining_prices_03a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_3a, 'Price'] < current_Q_A.loc[max_price_index_3a, 'Price']:
                        if min_price_index_3a < max_price_index_3a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_3a > max_price_index_3a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_3b = min(min_price_index_3a, max_price_index_3a)
                    larger_index_3b = max(min_price_index_3a, max_price_index_3a)
                    remaining_prices_03b = current_df[(current_df.index > smaller_index_3b) & (current_df.index < larger_index_3b)]
                    remaining_prices_03b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3b) & (Q_A_Preds.index < larger_index_3b)]
                    remaining_prices_03b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3b) & (Q_B_Preds.index < larger_index_3b)]

                    if len(remaining_prices_03b) > 1:
                        max_price_index_3b = remaining_prices_03b_A['Price'].idxmax()
                        min_price_index_3b= remaining_prices_03b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_3b, 'Price'] < current_Q_A.loc[max_price_index_3b, 'Price']:
                            if min_price_index_3b < max_price_index_3b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_3b > max_price_index_3b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)
                        
                                  prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_3c = min(min_price_index_3b, max_price_index_3b)
                        larger_index_3c = max(min_price_index_3b, max_price_index_3b)
                        remaining_prices_03c = current_df[(current_df.index > smaller_index_3c) & (current_df.index < larger_index_3c)]
                        remaining_prices_03c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3c) & (Q_A_Preds.index < larger_index_3c)]
                        remaining_prices_03c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3c) & (Q_B_Preds.index < larger_index_3c)]

                        if len(remaining_prices_03c) > 1:
                            max_price_index_3c = remaining_prices_03c_A['Price'].idxmax()
                            min_price_index_3c= remaining_prices_03c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_3c, 'Price'] < current_Q_A.loc[max_price_index_3c, 'Price']:
                                if min_price_index_3c < max_price_index_3c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_3c > max_price_index_3c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)
                            
                                      prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_3d = min(min_price_index_3c, max_price_index_3c)
                            larger_index_3d = max(min_price_index_3c, max_price_index_3c)
                            remaining_prices_03d = current_df[(current_df.index > smaller_index_3d) & (current_df.index < larger_index_3d)]
                            remaining_prices_03d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3d) & (Q_A_Preds.index < larger_index_3d)]
                            remaining_prices_03d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3d) & (Q_B_Preds.index < larger_index_3d)]

                            if len(remaining_prices_03d) > 1:
                                max_price_index_3d = remaining_prices_03d_A['Price'].idxmax()
                                min_price_index_3d= remaining_prices_03d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_3d, 'Price'] < current_Q_A.loc[max_price_index_3d, 'Price']:
                                    if min_price_index_3d < max_price_index_3d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_3d > max_price_index_3d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)
                            
                                          prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue                                            
                                            
                                                                                      
                                            
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)
PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)

In [35]:
print(sum(PF_DAM.iloc[:,4:5].values))
PF_DAM

[92952.72]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,4,11.58,16,31.66,20.08,3
1,23,13.98,17,31.66,17.68,3
2,22,14.26,18,28.66,14.40,3
3,21,20.36,19,23.22,2.86,3
4,5,12.26,9,25.83,13.57,3
...,...,...,...,...,...,...
2745,8759,83.68,8746,135.88,52.20,3
2746,8750,83.87,8747,135.56,51.69,3
2747,8749,90.00,8748,115.13,25.13,3
2748,8742,81.53,8744,121.10,39.57,3


In [36]:
print(sum(r_dam_50_50.iloc[:,4:5].values))
r_dam_50_50

[88780.19]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,5,12.26,17,31.66,19.40,3
1,23,13.98,18,28.66,14.68,3
2,22,14.26,19,23.22,8.96,3
3,21,20.36,20,21.00,0.64,3
4,14,15.16,10,25.67,10.51,3
...,...,...,...,...,...,...
2892,8758,87.55,8747,135.56,48.01,3
2893,8757,84.90,8753,111.42,26.52,3
2894,8756,94.50,8754,109.81,15.31,3
2895,8741,86.57,8743,108.00,21.43,3


In [37]:
print(sum(r_dam_10_90.iloc[:,4:5].values))
r_dam_10_90

[61272.06]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,25,12.64,42,36.00,23.36,3
1,26,12.88,41,39.82,26.94,3
2,27,10.26,34,20.66,10.40,3
3,28,12.89,33,23.13,10.24,3
4,50,16.23,66,37.30,21.07,3
...,...,...,...,...,...,...
1110,8710,87.92,8698,204.99,117.07,3
1111,8692,87.65,8695,258.87,171.22,3
1112,8715,77.62,8731,117.03,39.41,3
1113,8737,89.67,8744,121.10,31.43,3


In [38]:
print(sum(r_dam_30_70.iloc[:,4:5].values))
r_dam_30_70

[83150.31]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,4,11.58,18,28.66,17.08,3
1,23,13.98,19,23.22,9.24,3
2,22,14.26,20,21.00,6.74,3
3,14,15.16,17,31.66,16.50,3
4,15,24.69,16,31.66,6.97,3
...,...,...,...,...,...,...
2163,8714,76.97,8712,81.90,4.93,3
2164,8740,81.63,8744,121.10,39.47,3
2165,8759,83.68,8745,148.90,65.22,3
2166,8750,83.87,8746,135.88,52.01,3


In [39]:
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

In [40]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        
        
        

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                
                
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1a = min(min_price_index_1, max_price_index_1)
            larger_index_1a = max(min_price_index_1, max_price_index_1)
            remaining_prices_11a = current_df[(current_df.index > smaller_index_1a) & (current_df.index < larger_index_1a)]
            remaining_prices_11a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1a) & (Q_A_Preds.index < larger_index_1a)]
            remaining_prices_11a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1a) & (Q_B_Preds.index < larger_index_1a)]

            if len(remaining_prices_11a) > 1:
                max_price_index_1a = remaining_prices_11a_A['Price'].idxmax()
                min_price_index_1a = remaining_prices_11a_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_1a, 'Price'] < current_Q_A.loc[max_price_index_1a, 'Price']:
                    if min_price_index_1a < max_price_index_1a:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                    elif min_price_index_1a > max_price_index_1a:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                    else:
                         continue
                            
                            
                            
                            
                smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                remaining_prices_11b = current_df[(current_df.index > smaller_index_1b) & (current_df.index < larger_index_1b)]
                remaining_prices_11b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1b) & (Q_A_Preds.index < larger_index_1b)]
                remaining_prices_11b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1b) & (Q_B_Preds.index < larger_index_1b)]

                if len(remaining_prices_11b) > 1:
                    max_price_index_1b = remaining_prices_11b_A['Price'].idxmax()
                    min_price_index_1b = remaining_prices_11b_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_1b, 'Price'] < current_Q_A.loc[max_price_index_1b, 'Price']:
                        if min_price_index_1b < max_price_index_1b:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                        elif min_price_index_1b > max_price_index_1b:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                        else:
                             continue                           
                            
                            
                            
                    smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                    larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                    remaining_prices_11c = current_df[(current_df.index > smaller_index_1c) & (current_df.index < larger_index_1c)]
                    remaining_prices_11c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1c) & (Q_A_Preds.index < larger_index_1c)]
                    remaining_prices_11c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1c) & (Q_B_Preds.index < larger_index_1c)]

                    if len(remaining_prices_11c) > 1:
                        max_price_index_1c = remaining_prices_11c_A['Price'].idxmax()
                        min_price_index_1c = remaining_prices_11c_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_1c, 'Price'] < current_Q_A.loc[max_price_index_1c, 'Price']:
                            if min_price_index_1c < max_price_index_1c:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1c > max_price_index_1c:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                           
                                               
                                        
                                        
                                        
                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                        remaining_prices_11d = current_df[(current_df.index > smaller_index_1d) & (current_df.index < larger_index_1d)]
                        remaining_prices_11d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1d) & (Q_A_Preds.index < larger_index_1d)]
                        remaining_prices_11d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1d) & (Q_B_Preds.index < larger_index_1d)]

                        if len(remaining_prices_11d) > 1:
                            max_price_index_1d = remaining_prices_11d_A['Price'].idxmax()
                            min_price_index_1d = remaining_prices_11d_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_1d, 'Price'] < current_Q_A.loc[max_price_index_1d, 'Price']:
                                if min_price_index_1d < max_price_index_1d:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                elif min_price_index_1d > max_price_index_1d:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                      prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                 
                            
                            
                            
                            
                            
                            
                            
                            

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_02a = current_df[(current_df.index > smaller_index_2a) & (current_df.index < larger_index_2a)]
                remaining_prices_02a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2a) & (Q_A_Preds.index < larger_index_2a)]
                remaining_prices_02a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2a) & (Q_B_Preds.index < larger_index_2a)]

                if len(remaining_prices_02a) > 1:
                    max_price_index_2a = remaining_prices_02a_A['Price'].idxmax()
                    min_price_index_2a = remaining_prices_02a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_2a, 'Price'] < current_Q_A.loc[max_price_index_2a, 'Price']:
                        if min_price_index_2a < max_price_index_2a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_2a > max_price_index_2a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                    larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                    remaining_prices_02b = current_df[(current_df.index > smaller_index_2b) & (current_df.index < larger_index_2b)]
                    remaining_prices_02b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2b) & (Q_A_Preds.index < larger_index_2b)]
                    remaining_prices_02b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2b) & (Q_B_Preds.index < larger_index_2b)]

                    if len(remaining_prices_02b) > 1:
                        max_price_index_2b = remaining_prices_02b_A['Price'].idxmax()
                        min_price_index_2b= remaining_prices_02b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_2b, 'Price'] < current_Q_A.loc[max_price_index_2b, 'Price']:
                            if min_price_index_2b < max_price_index_2b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2b > max_price_index_2b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                        
                                  prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                        larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                        remaining_prices_02c = current_df[(current_df.index > smaller_index_2c) & (current_df.index < larger_index_2c)]
                        remaining_prices_02c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2c) & (Q_A_Preds.index < larger_index_2c)]
                        remaining_prices_02c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2c) & (Q_B_Preds.index < larger_index_2c)]

                        if len(remaining_prices_02c) > 1:
                            max_price_index_2c = remaining_prices_02c_A['Price'].idxmax()
                            min_price_index_2c= remaining_prices_02c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_2c, 'Price'] < current_Q_A.loc[max_price_index_2c, 'Price']:
                                if min_price_index_2c < max_price_index_2c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_2c > max_price_index_2c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                            
                                      prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                            larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                            remaining_prices_02d = current_df[(current_df.index > smaller_index_2d) & (current_df.index < larger_index_2d)]
                            remaining_prices_02d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2d) & (Q_A_Preds.index < larger_index_2d)]
                            remaining_prices_02d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2d) & (Q_B_Preds.index < larger_index_2d)]

                            if len(remaining_prices_02d) > 1:
                                max_price_index_2d = remaining_prices_02d_A['Price'].idxmax()
                                min_price_index_2d= remaining_prices_02d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_2d, 'Price'] < current_Q_A.loc[max_price_index_2d, 'Price']:
                                    if min_price_index_2d < max_price_index_2d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2d > max_price_index_2d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                            
                                          prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue  
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_3a = min(min_price_index_3, max_price_index_3)
                larger_index_3a = max(min_price_index_3, max_price_index_3)
                remaining_prices_03a = current_df[(current_df.index > smaller_index_3a) & (current_df.index < larger_index_3a)]
                remaining_prices_03a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3a) & (Q_A_Preds.index < larger_index_3a)]
                remaining_prices_03a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3a) & (Q_B_Preds.index < larger_index_3a)]

                if len(remaining_prices_03a) > 1:
                    max_price_index_3a = remaining_prices_03a_A['Price'].idxmax()
                    min_price_index_3a = remaining_prices_03a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_3a, 'Price'] < current_Q_A.loc[max_price_index_3a, 'Price']:
                        if min_price_index_3a < max_price_index_3a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_3a > max_price_index_3a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_3b = min(min_price_index_3a, max_price_index_3a)
                    larger_index_3b = max(min_price_index_3a, max_price_index_3a)
                    remaining_prices_03b = current_df[(current_df.index > smaller_index_3b) & (current_df.index < larger_index_3b)]
                    remaining_prices_03b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3b) & (Q_A_Preds.index < larger_index_3b)]
                    remaining_prices_03b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3b) & (Q_B_Preds.index < larger_index_3b)]

                    if len(remaining_prices_03b) > 1:
                        max_price_index_3b = remaining_prices_03b_A['Price'].idxmax()
                        min_price_index_3b= remaining_prices_03b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_3b, 'Price'] < current_Q_A.loc[max_price_index_3b, 'Price']:
                            if min_price_index_3b < max_price_index_3b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_3b > max_price_index_3b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)
                        
                                  prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_3c = min(min_price_index_3b, max_price_index_3b)
                        larger_index_3c = max(min_price_index_3b, max_price_index_3b)
                        remaining_prices_03c = current_df[(current_df.index > smaller_index_3c) & (current_df.index < larger_index_3c)]
                        remaining_prices_03c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3c) & (Q_A_Preds.index < larger_index_3c)]
                        remaining_prices_03c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3c) & (Q_B_Preds.index < larger_index_3c)]

                        if len(remaining_prices_03c) > 1:
                            max_price_index_3c = remaining_prices_03c_A['Price'].idxmax()
                            min_price_index_3c= remaining_prices_03c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_3c, 'Price'] < current_Q_A.loc[max_price_index_3c, 'Price']:
                                if min_price_index_3c < max_price_index_3c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_3c > max_price_index_3c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)
                            
                                      prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_3d = min(min_price_index_3c, max_price_index_3c)
                            larger_index_3d = max(min_price_index_3c, max_price_index_3c)
                            remaining_prices_03d = current_df[(current_df.index > smaller_index_3d) & (current_df.index < larger_index_3d)]
                            remaining_prices_03d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3d) & (Q_A_Preds.index < larger_index_3d)]
                            remaining_prices_03d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3d) & (Q_B_Preds.index < larger_index_3d)]

                            if len(remaining_prices_03d) > 1:
                                max_price_index_3d = remaining_prices_03d_A['Price'].idxmax()
                                min_price_index_3d= remaining_prices_03d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_3d, 'Price'] < current_Q_A.loc[max_price_index_3d, 'Price']:
                                    if min_price_index_3d < max_price_index_3d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_3d > max_price_index_3d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)
                            
                                          prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue                                            
                                            
                                                                                      
                                            
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])

r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
PF_BM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

# r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
# r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)


In [41]:
print(sum(PF_BM.iloc[:,4:5].values))
PF_BM

[233945.89]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,11,16.99,2,194.62,177.63,1
1,14,28.95,15,60.90,60.90,0
2,6,19.44,9,175.37,155.93,0
3,7,46.23,8,79.54,33.31,0
4,1,140.06,0,187.63,-140.06,1
...,...,...,...,...,...,...
4952,17508,39.54,17518,93.18,53.64,0
4953,17509,40.81,17517,79.96,39.15,0
4954,17510,53.13,17515,68.65,15.52,0
4955,17511,57.62,17512,67.63,10.01,0


In [42]:
print(sum(r_bm_50_50.iloc[:,4:5].values))
r_bm_50_50

[113977.67]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,13,33.28,0,187.63,154.35,1
1,14,28.95,15,60.90,60.90,0
2,11,16.99,5,100.34,-16.99,1
3,9,175.37,7,46.23,-129.14,1
4,31,14.21,22,45.66,31.45,1
...,...,...,...,...,...,...
6342,17505,38.58,17518,93.18,54.60,0
6343,17506,43.03,17517,79.96,36.93,0
6344,17507,36.82,17516,57.13,20.31,0
6345,17508,39.54,17515,68.65,29.11,0


In [43]:
print(sum(r_bm_10_90.iloc[:,4:5].values))
r_bm_10_90

[168.79]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,10634,211.54,10624,380.33,168.79,1


In [44]:
print(sum(r_bm_30_70.iloc[:,4:5].values))
r_bm_30_70

[26784.06]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,925,0.00,912,7.48,7.48,1
1,924,0.00,913,6.34,6.34,1
2,1277,5.86,1269,2.01,-3.85,1
3,1337,0.00,1343,0.29,0.29,0
4,1344,13.76,1359,32.01,18.25,0
...,...,...,...,...,...,...
370,17262,83.43,17250,159.56,76.13,1
371,17312,74.89,17327,171.28,171.28,0
372,17359,68.06,17345,187.11,-68.06,1
373,17504,42.38,17519,257.23,257.23,0


In [46]:
num_rows = 365 * 48
df = pd.DataFrame(index=range(num_rows))

# Add the 'level_0' column
df['level_0'] = np.repeat(np.arange(366), 48)[:num_rows]

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=pd.concat([dat1, df],axis=1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r_BM=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r_BM["Price"]=Y_r_BM.iloc[:,2:3]
Q_10_BM["Price"]=Q_10_BM.iloc[:,2:3]
Q_30_BM["Price"]=Q_30_BM.iloc[:,2:3]
Q_50_BM["Price"]=Q_50_BM.iloc[:,2:3]
Q_70_BM["Price"]=Q_70_BM.iloc[:,2:3]
Q_90_BM["Price"]=Q_90_BM.iloc[:,2:3]

Y_r_BM  = Y_r_BM.iloc[:,1:]
Q_10_BM = Q_10_BM.iloc[:,1:]
Q_30_BM = Q_30_BM.iloc[:,1:]
Q_50_BM = Q_50_BM.iloc[:,1:]
Q_70_BM = Q_70_BM.iloc[:,1:]
Q_90_BM = Q_90_BM.iloc[:,1:]

Y_r_BM= pd.concat([Y_r_BM, df],axis=1)
Q_10_BM= pd.concat([Q_10_BM, df],axis=1)
Q_30_BM= pd.concat([Q_30_BM, df],axis=1)
Q_50_BM= pd.concat([Q_50_BM, df],axis=1)
Q_70_BM= pd.concat([Q_70_BM, df],axis=1)
Q_90_BM= pd.concat([Q_90_BM, df],axis=1)

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)
start = 0
end = 17519
step = 2

values = list(range(start, end+1, step))
df1 = pd.DataFrame({'P_dam': values})

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r_DAM=dat1[column_names].dropna().stack().reset_index()
Y_r_DAM=Y_r_DAM.iloc[:,:]

Y_r_DAM["Price"]=Y_r_DAM.iloc[:,2:3]
Q_10_DAM["Price"]=Q_10_DAM.iloc[:,2:3]
Q_30_DAM["Price"]=Q_30_DAM.iloc[:,2:3]
Q_50_DAM["Price"]=Q_50_DAM.iloc[:,2:3]
Q_70_DAM["Price"]=Q_70_DAM.iloc[:,2:3]
Q_90_DAM["Price"]=Q_90_DAM.iloc[:,2:3]

Y_r_DAM= pd.concat([Y_r_DAM, df1], axis=1)
Q_10_DAM= pd.concat([Q_10_DAM, df1], axis=1)
Q_30_DAM= pd.concat([Q_30_DAM, df1], axis=1)
Q_50_DAM= pd.concat([Q_50_DAM, df1], axis=1)
Q_70_DAM= pd.concat([Q_70_DAM, df1], axis=1)
Q_90_DAM= pd.concat([Q_90_DAM, df1], axis=1)


In [67]:
def run_electricity_strategy(df, df_bm, Q_A_Preds, Q_B_Preds, Q_A_Preds_bm, Q_B_Preds_bm,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        current_df_bm = df_bm[df_bm['level_0'] == level_0]
        current_Q_A_bm = Q_A_Preds_bm[Q_A_Preds_bm['level_0'] == level_0]
        current_Q_B_bm = Q_B_Preds_bm[Q_B_Preds_bm['level_0'] == level_0]  
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()
        
        max_price_index = current_Q_A.loc[max_price_index, 'P_dam']
        min_price_index = current_Q_B.loc[min_price_index, 'P_dam']        

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        
        remaining_prices_0 = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]
        
        remaining_prices_1 = current_df_bm[(current_df_bm.index > smaller_index) & (current_df_bm.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index > larger_index)]
        remaining_prices_2 = current_df_bm[(current_df_bm.index < smaller_index) & (current_df_bm.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index < smaller_index) & (Q_A_Preds_bm.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index < smaller_index) & (Q_B_Preds_bm.index < larger_index)]

        if current_Q_B.loc[min_price_index/2, 'Price'] < current_Q_A.loc[max_price_index/2, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'], max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'],  max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                               
                        
        if len(remaining_prices_0) > 1:
            max_price_index_0 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_0 = remaining_prices_0_B['Price'].idxmin()            
            max_price_index_0 = remaining_prices_0_A.loc[max_price_index_0, 'P_dam']
            min_price_index_0 = remaining_prices_0_B.loc[min_price_index_0, 'P_dam']
            
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_0/2, 'Price'] < current_Q_A.loc[max_price_index_0/2, 'Price']:
                  if min_price_index_0 < max_price_index_0:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  elif min_price_index_0 > max_price_index_0:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                

            smaller_index_0a = min(min_price_index_0, max_price_index_0)
            larger_index_0a = max(min_price_index_0, max_price_index_0)
            remaining_prices_0a = current_df[(current_df['P_dam'] > smaller_index_0a) & (current_df['P_dam'] < larger_index_0a)]
            remaining_prices_0a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0a) & (Q_A_Preds['P_dam'] < larger_index_0a)]
            remaining_prices_0a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0a) & (Q_B_Preds['P_dam'] < larger_index_0a)]

            if len(remaining_prices_0a) > 1:

                    max_price_index_0a = remaining_prices_0a_A['Price'].idxmax()
                    min_price_index_0a = remaining_prices_0a_B['Price'].idxmin()
                    max_price_index_0a = remaining_prices_0a_A.loc[max_price_index_0a, 'P_dam']
                    min_price_index_0a = remaining_prices_0a_B.loc[min_price_index_0a, 'P_dam']
                    if current_Q_B.loc[min_price_index_0a/2, 'Price'] < current_Q_A.loc[max_price_index_0a/2, 'Price']:
                        if min_price_index_0a < max_price_index_0a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                    
                        elif min_price_index_0a > max_price_index_0a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                                
                        else:
                             continue     
                                
                                
                                
                                
                    smaller_index_0b = min(min_price_index_0a, max_price_index_0a)
                    larger_index_0b = max(min_price_index_0a, max_price_index_0a)
                    remaining_prices_0b = current_df[(current_df['P_dam'] > smaller_index_0b) & (current_df['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0b) & (Q_A_Preds['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0b) & (Q_B_Preds['P_dam'] < larger_index_0b)]

                    if len(remaining_prices_0b) > 1:

                            max_price_index_0b = remaining_prices_0b_A['Price'].idxmax()
                            min_price_index_0b = remaining_prices_0b_B['Price'].idxmin()
                            max_price_index_0b = remaining_prices_0b_A.loc[max_price_index_0b, 'P_dam']
                            min_price_index_0b = remaining_prices_0b_B.loc[min_price_index_0b, 'P_dam']
                            if current_Q_B.loc[min_price_index_0b/2, 'Price'] < current_Q_A.loc[max_price_index_0b/2, 'Price']:
                                if min_price_index_0b < max_price_index_0b:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                    
                                elif min_price_index_0b > max_price_index_0b:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                      prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                    
                                
                                
 
                            smaller_index_0c = min(min_price_index_0b, max_price_index_0b)
                            larger_index_0c = max(min_price_index_0b, max_price_index_0b)
                            remaining_prices_0c = current_df[(current_df['P_dam'] > smaller_index_0c) & (current_df['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0c) & (Q_A_Preds['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0c) & (Q_B_Preds['P_dam'] < larger_index_0c)]

                            if len(remaining_prices_0c) > 1:

                                    max_price_index_0c = remaining_prices_0c_A['Price'].idxmax()
                                    min_price_index_0c = remaining_prices_0c_B['Price'].idxmin()
                                    max_price_index_0c = remaining_prices_0c_A.loc[max_price_index_0c, 'P_dam']
                                    min_price_index_0c = remaining_prices_0c_B.loc[min_price_index_0c, 'P_dam']
                                    if current_Q_B.loc[min_price_index_0c/2, 'Price'] < current_Q_A.loc[max_price_index_0c/2, 'Price']:
                                        if min_price_index_0c < max_price_index_0c:
                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                            charge_level += bottleneck_1
                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                            charge_level -= bottleneck_2
                    
                                            profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                            prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                    
                                        elif min_price_index_0c > max_price_index_0c:
                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                              charge_level -= bottleneck_2
                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                              charge_level += bottleneck_1
                    
                                              profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                              prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                                
                                        else:
                                             continue


                                    smaller_index_0d = min(min_price_index_0c, max_price_index_0c)
                                    larger_index_0d = max(min_price_index_0c, max_price_index_0c)
                                    remaining_prices_0d = current_df[(current_df['P_dam'] > smaller_index_0d) & (current_df['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0d) & (Q_A_Preds['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0d) & (Q_B_Preds['P_dam'] < larger_index_0d)]

                                    if len(remaining_prices_0d) > 1:

                                            max_price_index_0d = remaining_prices_0d_A['Price'].idxmax()
                                            min_price_index_0d = remaining_prices_0d_B['Price'].idxmin()
                                            max_price_index_0d = remaining_prices_0d_A.loc[max_price_index_0d, 'P_dam']
                                            min_price_index_0d = remaining_prices_0d_B.loc[min_price_index_0d, 'P_dam']
                                            if current_Q_B.loc[min_price_index_0d/2, 'Price'] < current_Q_A.loc[max_price_index_0d/2, 'Price']:
                                                if min_price_index_0d < max_price_index_0d:
                                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                    charge_level += bottleneck_1
                                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                    charge_level -= bottleneck_2
                    
                                                    profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                    prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                    
                                                elif min_price_index_0d > max_price_index_0d:
                                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                      charge_level -= bottleneck_2
                                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                      charge_level += bottleneck_1
                    
                                                      profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                      prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                                
                                                else:
                                                     continue



                                            smaller_index_0e = min(min_price_index_0d, max_price_index_0d)
                                            larger_index_0e = max(min_price_index_0d, max_price_index_0d)
                                            remaining_prices_0e = current_df[(current_df['P_dam'] > smaller_index_0e) & (current_df['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0e) & (Q_A_Preds['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0e) & (Q_B_Preds['P_dam'] < larger_index_0e)]

                                            if len(remaining_prices_0e) > 1:

                                                    max_price_index_0e = remaining_prices_0e_A['Price'].idxmax()
                                                    min_price_index_0e = remaining_prices_0e_B['Price'].idxmin()
                                                    max_price_index_0e = remaining_prices_0e_A.loc[max_price_index_0e, 'P_dam']
                                                    min_price_index_0e = remaining_prices_0e_B.loc[min_price_index_0e, 'P_dam']
                                                    if current_Q_B.loc[min_price_index_0e/2, 'Price'] < current_Q_A.loc[max_price_index_0e/2, 'Price']:
                                                        if min_price_index_0e < max_price_index_0e:
                                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                            charge_level += bottleneck_1
                                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                            charge_level -= bottleneck_2
                    
                                                            profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                            prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                    
                                                        elif min_price_index_0e > max_price_index_0e:
                                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                              charge_level -= bottleneck_2
                                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                              charge_level += bottleneck_1
                    
                                                              profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                              prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                                
                                                        else:
                                                             continue









                                            
        if len(remaining_prices_1) > 1:

                max_price_index_1 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_1 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_1, 'Price'] < current_Q_A_bm.loc[max_price_index_1, 'Price']:
                      if min_price_index_1 < max_price_index_1:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      elif min_price_index_1 > max_price_index_1:
                             bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                             charge_level -= bottleneck_2
                             bottleneck_1 = min(capacity - charge_level, ramp_rate)
                             charge_level += bottleneck_1
            
                             profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                             prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      else:
                            continue                
                        

        
                smaller_index_1a = min(min_price_index_1, max_price_index_1)
                larger_index_1a = max(min_price_index_1, max_price_index_1)
                remaining_prices_1a = current_df_bm[(current_df_bm.index > smaller_index_1a) & (current_df_bm.index < larger_index_1a)]
                remaining_prices_1a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1a) & (Q_A_Preds_bm.index < larger_index_1a)]
                remaining_prices_1a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1a) & (Q_B_Preds_bm.index < larger_index_1a)]

                if len(remaining_prices_1a) > 1:
                        max_price_index_1a = remaining_prices_1a_A['Price'].idxmax()
                        min_price_index_1a = remaining_prices_1a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_1a, 'Price'] < current_Q_A_bm.loc[max_price_index_1a, 'Price']:
                            if min_price_index_1a < max_price_index_1a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1a > max_price_index_1a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                        larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                        remaining_prices_1b = current_df_bm[(current_df_bm.index > smaller_index_1b) & (current_df_bm.index < larger_index_1b)]
                        remaining_prices_1b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1b) & (Q_A_Preds_bm.index < larger_index_1b)]
                        remaining_prices_1b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1b) & (Q_B_Preds_bm.index < larger_index_1b)]

                        if len(remaining_prices_1b) > 1:
                                max_price_index_1b = remaining_prices_1b_A['Price'].idxmax()
                                min_price_index_1b = remaining_prices_1b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_1b, 'Price'] < current_Q_A_bm.loc[max_price_index_1b, 'Price']:
                                    if min_price_index_1b < max_price_index_1b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_1b > max_price_index_1b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                                          prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                                larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                                remaining_prices_1c = current_df_bm[(current_df_bm.index > smaller_index_1c) & (current_df_bm.index < larger_index_1c)]
                                remaining_prices_1c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1c) & (Q_A_Preds_bm.index < larger_index_1c)]
                                remaining_prices_1c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1c) & (Q_B_Preds_bm.index < larger_index_1c)]

                                if len(remaining_prices_1c) > 1:
                                        max_price_index_1c = remaining_prices_1c_A['Price'].idxmax()
                                        min_price_index_1c = remaining_prices_1c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_1c, 'Price'] < current_Q_A_bm.loc[max_price_index_1c, 'Price']:
                                            if min_price_index_1c < max_price_index_1c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                    
                                                profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_1c > max_price_index_1c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                    
                                                  profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                  prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                                        remaining_prices_1d = current_df_bm[(current_df_bm.index > smaller_index_1d) & (current_df_bm.index < larger_index_1d)]
                                        remaining_prices_1d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1d) & (Q_A_Preds_bm.index < larger_index_1d)]
                                        remaining_prices_1d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1d) & (Q_B_Preds_bm.index < larger_index_1d)]
    
                                        if len(remaining_prices_1d) > 1:
                                                max_price_index_1d = remaining_prices_1d_A['Price'].idxmax()
                                                min_price_index_1d = remaining_prices_1d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_1d, 'Price'] < current_Q_A_bm.loc[max_price_index_1d, 'Price']:
                                                    if min_price_index_1d < max_price_index_1d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                    
                                                        profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                        prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_1d > max_price_index_1d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                    
                                                          profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                          prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_1e = min(min_price_index_1d, max_price_index_1d)
                                                larger_index_1e = max(min_price_index_1d, max_price_index_1d)
                                                remaining_prices_1e = current_df_bm[(current_df_bm.index > smaller_index_1e) & (current_df_bm.index < larger_index_1e)]
                                                remaining_prices_1e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1e) & (Q_A_Preds_bm.index < larger_index_1e)]
                                                remaining_prices_1e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1e) & (Q_B_Preds_bm.index < larger_index_1e)]
    
                                                if len(remaining_prices_1e) > 1:
                                                        max_price_index_1e = remaining_prices_1e_A['Price'].idxmax()
                                                        min_price_index_1e = remaining_prices_1e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_1e, 'Price'] < current_Q_A_bm.loc[max_price_index_1e, 'Price']:
                                                            if min_price_index_1e < max_price_index_1e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                    
                                                                profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_1e > max_price_index_1e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                    
                                                                  profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                  prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                          
                                        
                                        
                                        
                                        
                                        
                       
                        
                        
        if len(remaining_prices_2) > 1:
                max_price_index_2 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_2, 'Price'] < current_Q_A_bm.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue  
                        
                        
                      
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_2a = current_df_bm[(current_df_bm.index > smaller_index_2a) & (current_df_bm.index < larger_index_2a)]
                remaining_prices_2a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2a) & (Q_A_Preds_bm.index < larger_index_2a)]
                remaining_prices_2a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2a) & (Q_B_Preds_bm.index < larger_index_2a)]

                if len(remaining_prices_2a) > 1:
                        max_price_index_2a = remaining_prices_2a_A['Price'].idxmax()
                        min_price_index_2a = remaining_prices_2a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_2a, 'Price'] < current_Q_A_bm.loc[max_price_index_2a, 'Price']:
                            if min_price_index_2a < max_price_index_2a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2a > max_price_index_2a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                        larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                        remaining_prices_2b = current_df_bm[(current_df_bm.index > smaller_index_2b) & (current_df_bm.index < larger_index_2b)]
                        remaining_prices_2b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2b) & (Q_A_Preds_bm.index < larger_index_2b)]
                        remaining_prices_2b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2b) & (Q_B_Preds_bm.index < larger_index_2b)]

                        if len(remaining_prices_2b) > 1:
                                max_price_index_2b = remaining_prices_2b_A['Price'].idxmax()
                                min_price_index_2b = remaining_prices_2b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_2b, 'Price'] < current_Q_A_bm.loc[max_price_index_2b, 'Price']:
                                    if min_price_index_2b < max_price_index_2b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2b > max_price_index_2b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                          prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                                larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                                remaining_prices_2c = current_df_bm[(current_df_bm.index > smaller_index_2c) & (current_df_bm.index < larger_index_2c)]
                                remaining_prices_2c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2c) & (Q_A_Preds_bm.index < larger_index_2c)]
                                remaining_prices_2c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2c) & (Q_B_Preds_bm.index < larger_index_2c)]

                                if len(remaining_prices_2c) > 1:
                                        max_price_index_2c = remaining_prices_2c_A['Price'].idxmax()
                                        min_price_index_2c = remaining_prices_2c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_2c, 'Price'] < current_Q_A_bm.loc[max_price_index_2c, 'Price']:
                                            if min_price_index_2c < max_price_index_2c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                    
                                                profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_2c > max_price_index_2c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                    
                                                  profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                  prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                                        larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                                        remaining_prices_2d = current_df_bm[(current_df_bm.index > smaller_index_2d) & (current_df_bm.index < larger_index_2d)]
                                        remaining_prices_2d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2d) & (Q_A_Preds_bm.index < larger_index_2d)]
                                        remaining_prices_2d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2d) & (Q_B_Preds_bm.index < larger_index_2d)]
    
                                        if len(remaining_prices_2d) > 1:
                                                max_price_index_2d = remaining_prices_2d_A['Price'].idxmax()
                                                min_price_index_2d = remaining_prices_2d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_2d, 'Price'] < current_Q_A_bm.loc[max_price_index_2d, 'Price']:
                                                    if min_price_index_2d < max_price_index_2d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                    
                                                        profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                        prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_2d > max_price_index_2d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                    
                                                          profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                          prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_2e = min(min_price_index_2d, max_price_index_2d)
                                                larger_index_2e = max(min_price_index_2d, max_price_index_2d)
                                                remaining_prices_2e = current_df_bm[(current_df_bm.index > smaller_index_2e) & (current_df_bm.index < larger_index_2e)]
                                                remaining_prices_2e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2e) & (Q_A_Preds_bm.index < larger_index_2e)]
                                                remaining_prices_2e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2e) & (Q_B_Preds_bm.index < larger_index_2e)]
    
                                                if len(remaining_prices_2e) > 1:
                                                        max_price_index_2e = remaining_prices_2e_A['Price'].idxmax()
                                                        min_price_index_2e = remaining_prices_2e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_2e, 'Price'] < current_Q_A_bm.loc[max_price_index_2e, 'Price']:
                                                            if min_price_index_2e < max_price_index_2e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                    
                                                                profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_2e > max_price_index_2e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                    
                                                                  profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                  prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                         
                                                                      

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_bm_50_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_90_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_70_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
PF_dam_bm=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Y_r_DAM, Q_B_Preds=Y_r_DAM, Q_A_Preds_bm=Y_r_BM, Q_B_Preds_bm=Y_r_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)


In [69]:
print(sum(PF_dam_bm.iloc[:,4:5].values))
PF_dam_bm.head(n=10)

[143829.91]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,8,11.58,32,31.66,31.66,0
1,10,12.26,18,25.83,13.57,0
2,12,15.34,16,23.22,7.88,0
3,42,7.66,47,27.74,20.08,0
4,43,8.94,46,22.55,13.61,0
5,44,11.75,45,12.69,0.94,0
6,6,19.44,2,194.62,-19.44,1
7,4,30.91,46,100.34,100.34,0
8,54,10.26,82,39.82,29.56,0
9,56,12.89,80,31.55,18.66,0


In [70]:
print(sum(r_dam_bm_50_50.iloc[:,4:5].values))
r_dam_bm_50_50

[103460.61]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,10,12.26,34,31.66,31.66,0
1,28,15.16,20,25.67,-15.16,1
2,26,13.92,22,23.83,9.91,1
3,35,8.05,47,27.74,27.74,0
4,36,17.02,45,12.69,-4.33,0
...,...,...,...,...,...,...
3779,17507,36.82,17515,68.65,31.83,0
3780,17508,39.54,17514,61.67,22.13,0
3781,17479,34.71,17473,59.17,-34.71,1
3782,17478,55.64,17474,57.74,2.10,1


In [71]:
print(sum(r_dam_bm_10_90.iloc[:,4:5].values))
r_dam_bm_10_90

[46547.88]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,50,12.64,84,36.00,36.00,0
1,52,12.88,82,39.82,26.94,0
2,54,10.26,68,20.66,10.40,0
3,56,12.89,66,23.13,10.24,0
4,100,16.23,132,37.30,21.07,0
...,...,...,...,...,...,...
893,17334,86.24,17344,246.11,159.87,0
894,17382,85.31,17392,252.37,167.06,0
895,17384,87.65,17390,258.87,171.22,0
896,17430,77.62,17462,117.03,39.41,0


In [72]:
print(sum(r_dam_bm_30_70.iloc[:,4:5].values))
r_dam_bm_30_70

[69223.66]


,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit,charge Level
0,8,11.58,36,28.66,28.66,0
1,28,15.16,34,31.66,16.50,0
2,30,24.69,32,31.66,6.97,0
3,50,12.64,84,36.00,23.36,0
4,52,12.88,82,39.82,26.94,0
...,...,...,...,...,...,...
1552,17452,83.86,17458,121.35,37.49,0
1553,17480,81.63,17488,121.10,39.47,0
1554,17482,86.57,17486,108.00,21.43,0
1555,17489,54.08,17519,257.23,203.15,0


In [ ]:
def run_electricity_strategy(df, df_bm, Q_A_Preds, Q_B_Preds, Q_A_Preds_bm, Q_B_Preds_bm,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        current_df_bm = df_bm[df_bm['level_0'] == level_0]
        current_Q_A_bm = Q_A_Preds_bm[Q_A_Preds_bm['level_0'] == level_0]
        current_Q_B_bm = Q_B_Preds_bm[Q_B_Preds_bm['level_0'] == level_0]  
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()
        
        max_price_index = current_Q_A.loc[max_price_index, 'P_dam']
        min_price_index = current_Q_B.loc[min_price_index, 'P_dam']        

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        
        remaining_prices_0 = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]
        
        remaining_prices_1 = current_df_bm[(current_df_bm.index > smaller_index) & (current_df_bm.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds_bm[(Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds_bm[(Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index > larger_index)]
        remaining_prices_2 = current_df_bm[(current_df_bm.index < smaller_index) & (current_df_bm.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds_bm[(Q_A_Preds_bm.index < smaller_index) & (Q_A_Preds_bm.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds_bm[(Q_B_Preds_bm.index < smaller_index) & (Q_B_Preds_bm.index < larger_index)]

        if current_Q_B.loc[min_price_index/2, 'Price'] < current_Q_A.loc[max_price_index/2, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'], max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'],  max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                                    
      
        if len(remaining_prices_1) > 1:
            try:

                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_2, 'Price'] < current_Q_A_bm.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                             bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                             charge_level -= bottleneck_2
                             bottleneck_1 = min(capacity - charge_level, ramp_rate)
                             charge_level += bottleneck_1
            
                             profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                             prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
            except KeyError:
                continue                        
                        
                        
        if len(remaining_prices_2) > 1:
            try:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_3, 'Price'] < current_Q_A_bm.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_3, current_df_bm.loc[min_price_index_3, 'Price'], max_price_index_3, current_df_bm.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
            
                            profit = (current_df_bm.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_3, current_df_bm.loc[min_price_index_3, 'Price'], max_price_index_3, current_df_bm.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue  
                        
                        
            except KeyError:
                continue                        
                        
                        
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()            
            max_price_index_1 = remaining_prices_0_A.loc[max_price_index_1, 'P_dam']
            min_price_index_1 = remaining_prices_0_B.loc[min_price_index_1, 'P_dam']
            
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1/2, 'Price'] < current_Q_A.loc[max_price_index_1/2, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1/2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1/2, 'Price'], max_price_index_1, current_df.loc[max_price_index_1/2, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1/2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1/2, 'Price'], max_price_index_1, current_df.loc[max_price_index_1/2, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                

            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df['P_dam'] > smaller_index_1) & (current_df['P_dam'] < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds['P_dam'] > smaller_index_1) & (Q_A_Preds['P_dam'] < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds['P_dam'] > smaller_index_1) & (Q_B_Preds['P_dam'] < larger_index_1)]

            if len(remaining_prices_01) > 1:
                try:

                    max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                    min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                    max_price_index_4 = remaining_prices_01_A.loc[max_price_index_4, 'P_dam']
                    min_price_index_4 = remaining_prices_01_B.loc[min_price_index_4, 'P_dam']
                    if current_Q_B.loc[min_price_index_4/2, 'Price'] < current_Q_A.loc[max_price_index_4/2, 'Price']:
                        if min_price_index_4 < max_price_index_4:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_4/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4/2, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                        elif min_price_index_4 > max_price_index_4:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_4/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1/2, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_4, current_df.loc[min_price_index_4/2, 'Price'], max_price_index_4, current_df.loc[max_price_index_4/2, 'Price'], profit, charge_level))
                                
                        else:
                             continue                     
                except KeyError:
                     continue
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_bm_50_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_90_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_70_BM, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)


In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
                    

                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



In [ ]:
        if len(remaining_prices_1) > 1:
            try:

                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
            except KeyError:
                continue

In [ ]:
        if len(remaining_prices_1) > 1:
            try:

                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
            min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
            if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                     bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                     charge_level -= bottleneck_2
                     bottleneck_1 = min(capacity - charge_level, ramp_rate)
                     charge_level += bottleneck_1
            
                     profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                     prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
print(sum(run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0).iloc[:,4:5].values))

In [ ]:
#DAM

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue


#         if len(remaining_prices_1) > 12:
# #             try:

#                 max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
#                 min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
#                 if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
#                       if min_price_index_2 < max_price_index_2:
#                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
#                         charge_level += bottleneck_1
#                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
#                         charge_level -= bottleneck_2
            
#                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
#                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
#                       elif min_price_index_2 > max_price_index_2:
#                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
#                          charge_level -= bottleneck_2
#                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
#                          charge_level += bottleneck_1
            
#                          profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
#                          prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
#                       else:
#                             continue
# #             except KeyError:
# #                 continue
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=1, eff_2=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
            min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
            if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                     bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                     charge_level -= bottleneck_2
                     bottleneck_1 = min(capacity - charge_level, ramp_rate)
                     charge_level += bottleneck_1
            
                     profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                     prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            remaining_prices_01_A = Q_A_Preds[(Q_A_Preds.index > smaller_index_1) & (Q_A_Preds.index < larger_index_1)]
            remaining_prices_01_B = Q_B_Preds[(Q_B_Preds.index > smaller_index_1) & (Q_B_Preds.index < larger_index_1)]

            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01_A['Price'].idxmax()
                min_price_index_4 = remaining_prices_01_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_4, 'Price'] < current_Q_A.loc[max_price_index_4, 'Price']:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                  
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
            min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
            if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
            min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
            if current_Q_A.loc[min_price_index_3, 'Price'] < current_Q_B.loc[max_price_index_3, 'Price']:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df  = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]

        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

#         smaller_index = min(min_price_index, max_price_index)
#         larger_index = max(min_price_index, max_price_index)
        
#         remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
#         remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
#         remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue                   
   

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
print(sum(run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0).iloc[:,4:5].values))

In [ ]:
def run_electricity_strategy(df, eff, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            charge_level=charge_level
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01['Price'].idxmax()
                min_price_index_4 = remaining_prices_01['Price'].idxmin()
                if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_4, 'Price'] * eff:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                    
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_2, 'Price'] * eff:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, eff=1, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

In [ ]:
print(sum(run_electricity_strategy(df=Y_r, eff=1, capacity=4,charge_level=1, ramp_rate=1, min_charge_level=0).iloc[:,4:5].values))

In [ ]:
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]


In [ ]:
Y_r.head(n=40)

In [ ]:
def run_electricity_strategy(df, eff, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            charge_level=charge_level
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01['Price'].idxmax()
                min_price_index_4 = remaining_prices_01['Price'].idxmin()
                if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_4, 'Price'] * eff:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                    
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_2, 'Price'] * eff:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, eff=1, capacity=10,charge_level=4, ramp_rate=4, min_charge_level=1)

In [ ]:
print(sum(run_electricity_strategy(df=Y_r, eff=1, capacity=10,charge_level=10, ramp_rate=10, min_charge_level=0).iloc[:,4:5].values))

In [ ]:
plt.plot(run_electricity_strategy(df=Y_r, eff=1, capacity=10,charge_level=10, ramp_rate=10, min_charge_level=0).iloc[:,4:5].values)

In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 1    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  
       

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 5
    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        if min_price_index < max_price_index:
            bottleneck_1 = min(capacity - charge_level, ramp_rate)
            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
            charge_level += bottleneck_1
            charge_level -= bottleneck_2
            
            profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], 
                           max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
        elif min_price_index > max_price_index:
            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
            charge_level -= bottleneck_2
            bottleneck_1 = min(capacity - charge_level, ramp_rate)
            charge_level += bottleneck_1
            
            profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], 
                           max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
        else:
            continue

    return prices
run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 5

    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]

        if len(remaining_prices_0) > 1:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                max_price = current_df.loc[max_price_index, 'Price']
                min_price = current_df.loc[min_price_index, 'Price']
                profit = (max_price * bottleneck_2 * eff) - (min_price * bottleneck_1)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, max_price, bottleneck_1, bottleneck_2, profit, charge_level))
                
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                max_price_1 = remaining_prices_0['Price'].max()
                min_price_1 = remaining_prices_0['Price'].min()
                profit_1 = (max_price_1 * bottleneck_2 * eff) - (min_price_1 * bottleneck_1)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], remaining_prices_0['Price'].idxmax(), max_price_1, bottleneck_1, bottleneck_2, profit_1, charge_level))
                
            else:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                max_price_1 = remaining_prices_0['Price'].max()
                min_price_1 = remaining_prices_0['Price'].min()
                profit_1 = (max_price_1 * bottleneck_2 * eff) - (min_price_1 * bottleneck_1)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], bottleneck_1, bottleneck_2, profit_1, charge_level))
                
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                max_price = current_df.loc[max_price_index, 'Price']
                min_price = current_df.loc[min_price_index, 'Price']
                profit = (max_price * bottleneck_2 * eff) - (min_price * bottleneck_1)
                prices.append((remaining_prices_0['Price'].idxmin(), remaining_prices_0['Price'].min(), max_price_index, current_df.loc[max_price


run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [ ]:

profitSpecific(maxHourIndex)=(price(maxHourIndex)*eff-price(minHourIndex)); 
profitSpecific(minHourIndex)=-profitSpecific(maxHourIndex)

if price(maxHourIndex)*eff> price(minHourIndex)
    if minHourIndex<maxHourIndex 
        storageFree=capacity-max(chargeLevel(minHourIndex:maxHourIndex));
        bottleneck=min([TAvail(maxHourIndex),PAvail(minHourIndex),storageFree/periodT]); 
            
    else if minHourIndex>maxHourIndex 
        storageLeft=min(chargeLevel(maxHourIndex:minHourIndex));
        bottleneck=min([TAvail(maxHourIndex),PAvail(minHourIndex),storageLeft/periodT]); 
    end
        
        
TAvail(maxHourIndex)=TAvail(maxHourIndex)-bottleneck;   
PAvail(minHourIndex)=PAvail(minHourIndex)-bottleneck;   
Tuse(maxHourIndex)=Tuse(maxHourIndex)+bottleneck;       
Puse(minHourIndex)=Puse(minHourIndex)+bottleneck;      
        
chargeLevel(maxHourIndex+1:end)=chargeLevel(maxHourIndex+1:end)-bottleneck*periodT;
chargeLevel(minHourIndex+1:end)=chargeLevel(minHourIndex+1:end)+bottleneck*periodT;

profit(maxHourIndex)=Tuse(maxHourIndex)*periodT*profitSpecific(maxHourIndex);  
profit(minHourIndex)=-profit(maxHourIndex);

profitTotal=profitTotal+profit(maxHourIndex);


In [ ]:
def run_electricity_strategy(df, eff, capacity, ramp_rate, min_charge_level):
    prices = []
    charge_level = 5    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if current_df.loc[min_price_index, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index, 'Price'] * bottleneck_1)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
            
            
            
            
           
            
            
            
            
            
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    
                    
                    
 
                    
            smaller_index_1 = min(min_price_index_1, max_price_index_1)
            larger_index_1 = max(min_price_index_1, max_price_index_1)
            remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
            if len(remaining_prices_01) > 1:
                max_price_index_4 = remaining_prices_01['Price'].idxmax()
                min_price_index_4 = remaining_prices_01['Price'].idxmin()
                if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_4, 'Price'] * eff:
                    if min_price_index_4 < max_price_index_4:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_4, 'Price'] * bottleneck_1)
                    
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_1, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                    
                    elif min_price_index_4 > max_price_index_4:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_4, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)
                    
                          prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], max_price_index_4, current_df.loc[max_price_index_4, 'Price'], profit, charge_level))
                                
                    else:
                         continue

                    
                    
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_2, 'Price'] * eff:
                  if min_price_index_2 < max_price_index_2:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  elif min_price_index_2 > max_price_index_2:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                  else:
                        continue
                    

                    
                    
        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                  if min_price_index_3 < max_price_index_3:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                    prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  elif min_price_index_3 > max_price_index_3:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff) - (current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                  else:
                        continue         
                        
    
    

    return prices
run_electricity_strategy(df=Y_r, eff=1, capacity=5, ramp_rate=2, min_charge_level=1)

In [ ]:
def run_electricity_strategy(df, eff):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        max_price_index = current_df['Price'].idxmax()
        min_price_index = current_df['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]

        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0['Price'].idxmax()
            min_price_index_1 = remaining_prices_0['Price'].idxmin()
            if current_df.loc[min_price_index_1, 'Price'] < current_df.loc[max_price_index, 'Price'] * eff:
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], 
                               max_price_index, current_df.loc[max_price_index, 'Price']))

                prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], 
                               max_price_index_1, current_df.loc[max_price_index_1, 'Price']))

                smaller_index_1 = min(min_price_index_1, max_price_index_1)
                larger_index_1 = max(min_price_index_1, max_price_index_1)
                remaining_prices_01 = current_df[(current_df.index > smaller_index_1) & (current_df.index < larger_index_1)]
                if len(remaining_prices_01) > 1:
                    max_price_index_4 = remaining_prices_01['Price'].idxmax()
                    min_price_index_4 = remaining_prices_01['Price'].idxmin()
                    if current_df.loc[min_price_index_4, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                        prices.append((min_price_index_4, current_df.loc[min_price_index_4, 'Price'], 
                                       max_price_index_4, current_df.loc[max_price_index_4, 'Price']))           
        if len(remaining_prices_1) > 1:
            max_price_index_2 = remaining_prices_1['Price'].idxmax()
            min_price_index_2 = remaining_prices_1['Price'].idxmin()
            if current_df.loc[min_price_index_2, 'Price'] < current_df.loc[max_price_index_1, 'Price'] * eff:
                prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], 
                               max_price_index_2, current_df.loc[max_price_index_2, 'Price']))

        if len(remaining_prices_2) > 1:
            max_price_index_3 = remaining_prices_2['Price'].idxmax()
            min_price_index_3 = remaining_prices_2['Price'].idxmin()
            if current_df.loc[min_price_index_3, 'Price'] < current_df.loc[max_price_index_3, 'Price'] * eff:
                prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], 
                               max_price_index_3, current_df.loc[max_price_index_3, 'Price']))

                               
    return prices

run_electricity_strategy(df=Y_r, eff=1)